In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# Загружаем набор данных из файла
# Первая часть тренировочная, а на второй нейросеть тестируется
training_set = pd.read_csv('data.csv')
training_set = training_set.iloc[:,1:2].values
plt.plot(training_set, linewidth=1.5)
plt.grid()
plt.show()

In [ ]:
# Функция алгоритма "скользящего окна"
def sliding_windows(data, seq_length):
    x = []
    y = []

    for i in range(len(data)-seq_length-1):
        _x = data[i:(i+seq_length)]
        _y = data[i+seq_length]
        x.append(_x)
        y.append(_y)

    return np.array(x), np.array(y)

# Производим стандартизацию данных
sc = MinMaxScaler()
training_data = sc.fit_transform(training_set)

seq_length = 4
x, y = sliding_windows(training_data, seq_length)

# Задаем размеры для данных тренировки и теста (по 50%)
train_size = int(len(y) * 0.5)
test_size = len(y) - train_size

# Делаем из данных тензоры
dataX = Variable(torch.Tensor(np.array(x)))
dataY = Variable(torch.Tensor(np.array(y)))

trainX = Variable(torch.Tensor(np.array(x[0:train_size])))
trainY = Variable(torch.Tensor(np.array(y[0:train_size])))

testX = Variable(torch.Tensor(np.array(x[train_size:len(x)])))
testY = Variable(torch.Tensor(np.array(y[train_size:len(y)])))

In [ ]:
# Описание нейронной сети долгой краткосрочной памяти
class LSTM(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()

        # Количество выходных параметров
        self.num_classes = num_classes
        # Количество рекурентных слоев
        self.num_layers = num_layers
        # Количестов входных параметров
        self.input_size = input_size
        # Количетсво скрытых параметров
        self.hidden_size = hidden_size

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)

        # Применяем линейное преобразование к входящим данным
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size))
        c_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size))

        # Пробрасываем данные через LSTM
        ula, (h_out, _) = self.lstm(x, (h_0, c_0))

        h_out = h_out.view(-1, self.hidden_size)

        out = self.fc(h_out)

        return out

In [ ]:
# Количество эпох
num_epochs = 4000
# Коэффициент скорости обучения
learning_rate = 0.01

# Базовые параметры сети
input_size = 1
hidden_size = 2
num_layers = 1
num_classes = 1

lstm = LSTM(num_classes, input_size, hidden_size, num_layers)

# Среднеквадратическая ошибка для регрессии
criterion = torch.nn.MSELoss()
# Оптимизация с применением алгоритма Адама
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)

# Тренируем модель
for epoch in range(num_epochs):
    outputs = lstm(trainX)
    optimizer.zero_grad()
    
    # Получаем функцию потерь
    loss = criterion(outputs, trainY)
    
    loss.backward()
    
    optimizer.step()

In [ ]:
# Делаем предсказания по dataX
lstm.eval()
train_predict = lstm(dataX)

data_predict = train_predict.data.numpy()
dataY_plot = dataY.data.numpy()

# Результат работы. Тут произведенно обратно скалирование
data_predict = sc.inverse_transform(data_predict)

# Рисуем график (красная линия разделяет train и test) 
dataY_plot = sc.inverse_transform(dataY_plot)
plt.axvline(x=train_size, c='r', linestyle='--')
plt.plot(dataY_plot, linewidth=1.5)
plt.plot(data_predict, linewidth=1.5)
plt.grid()
plt.show()

In [ ]:
import time
import sys
import numpy as np
import vrep
from assets import *

In [ ]:
class Algorithm:
  def __init__(self, ROBOT, TARGET):
    # Устанавливаем состояние движения
    self.state = States.MOVING

    # Минимальное и максимальное расстояние до препятствия
    self.MIN_DETECTION_DIST = 0
    self.MAX_DETECTION_DIST = 1

    # Название робота на сцене
    self.ROBOT = ROBOT
    # Название цели на сцене
    self.TARGET = TARGET
    # Скорость робота
    self.SPEED = 1.5
    # Отступ вокруг робота
    self.INDENT_DIST = 0.5

    # Задержка между выполнениями итераций цикла
    self.SLEEP_TIME = 0.2
    # Число "пи"
    self.PI = math.pi

    # Инициализация направления, позиции и значений углов Эйлера для работа
    self.bot_dir = None
    self.bot_pos = None
    self.bot_euler_angles = None

    # Инициализация позиции цели и направления до нее
    self.target_pos = None
    self.target_dir = np.zeros(3)

    # Инициализация массива значений угла поворота
    self.angle_arr = []

    # Инициализация массива значений сенсоров
    self.detect = np.zeros(16)

    # Инициализация подключения к серверу V-REP
    self._init_client_id()
    #  Инициализация хендла робота
    self._init_robot_handle()
    #  Инициализация хендлов сенсора робота
    self._init_sensor_handles()
    #  Инициализация хендлов колес робота
    self._init_wheels_handles()
    #  Инициализация хендла цели
    self._init_target_handle()

    # Инициализация частот и коэффициентов ПИД-регуляторов объезда препятствия
    self.obstacle_dist_stab_PID = PIDController(50)
    self.obstacle_follower_PID = PIDController(50)
    self.obstacle_dist_stab_PID.set_coefficients(2, 0, 0.5)
    self.obstacle_follower_PID.set_coefficients(2, 0, 0)

    # Инициализация значения ммнимальной дистанции до цели
    self.min_dist_to_target = None

    # Инициализация предсказанного значения до цели
    self.current_dist_to_target_predict = 0

    # Вывод информации
    self.print_info()


  def _init_client_id(self):
    vrep.simxFinish(-1)

    self.client_id = vrep.simxStart('127.0.0.1', 19999, True, True, 5000, 5)

    if self.client_id != -1:
      print('')
    else:
      sys.exit('')


  def _init_robot_handle(self):
    error_code, self.bot_handle = vrep.simxGetObjectHandle(self.client_id, self.ROBOT, vrep.simx_opmode_oneshot_wait)


  def _init_sensor_handles(self):
    self.sensor_handles = []

    for x in range(0, 16):
      error_code, sensor_handle = vrep.simxGetObjectHandle(self.client_id, 'Pioneer_p3dx_ultrasonicSensor' + str(x + 1), vrep.simx_opmode_oneshot_wait)
      self.sensor_handles.append(sensor_handle)
      vrep.simxReadProximitySensor(self.client_id, sensor_handle, vrep.simx_opmode_streaming)


  def _init_wheels_handles(self):
    error_code, self.left_motor_handle = vrep.simxGetObjectHandle(self.client_id, 'Pioneer_p3dx_leftMotor', vrep.simx_opmode_oneshot_wait)
    error_code, self.right_motor_handle = vrep.simxGetObjectHandle(self.client_id, 'Pioneer_p3dx_rightMotor', vrep.simx_opmode_oneshot_wait)


  def _init_target_handle(self):
    error_code, self.target_handle = vrep.simxGetObjectHandle(self.client_id, self.TARGET, vrep.simx_opmode_oneshot_wait)


  def _init_values(self):
    # Инициализация значений цели и робота
    error_code, _ = vrep.simxGetObjectPosition(self.client_id, self.target_handle, -1, vrep.simx_opmode_oneshot)
    error_code, _ = vrep.simxGetObjectPosition(self.client_id, self.bot_handle, -1, vrep.simx_opmode_oneshot)
    error_code, _ = vrep.simxGetObjectOrientation(self.client_id, self.bot_handle, -1, vrep.simx_opmode_streaming)


  def read_values(self):
    # Чтение значений позиции цели
    error_code, target_pos = vrep.simxGetObjectPosition(self.client_id, self.target_handle, -1, vrep.simx_opmode_streaming)
    self.target_pos = Vector3(x=target_pos[0], y=target_pos[1], z=0)

    # Чтение значений позиции робота
    error_code, bot_pos = vrep.simxGetObjectPosition(self.client_id, self.bot_handle, -1, vrep.simx_opmode_streaming)
    self.bot_pos = Vector3(x=bot_pos[0], y=bot_pos[1], z=0)

    # Чтение значений углов Эйлера робота
    error_code, bot_euler_angles = vrep.simxGetObjectOrientation(self.client_id, self.bot_handle, -1, vrep.simx_opmode_streaming)
    self.bot_euler_angles = Vector3(x=0, y=0, z=bot_euler_angles[2])


  def stop_move(self):
    # Остановка движения робота
    error_code = vrep.simxSetJointTargetVelocity(self.client_id, self.left_motor_handle,  0, vrep.simx_opmode_streaming)
    error_code = vrep.simxSetJointTargetVelocity(self.client_id, self.right_motor_handle, 0, vrep.simx_opmode_streaming)


  def read_from_sensors(self):
    # Чтение значений с сенсоров
    for i in range(0, 16):
      error_code, detection_state, detected_point, detected_object_handle, detected_surface_normal_vector = vrep.simxReadProximitySensor(self.client_id, self.sensor_handles[i], vrep.simx_opmode_streaming)

      # Значение дистанции до препятствия
      dist = math.sqrt(detected_point[0] ** 2 + detected_point[1] ** 2 + detected_point[2] ** 2)

      # Присвоение значений датчиков: чем больше значение, тем ближе к конкретному датчику препятствие
      if dist < self.MIN_DETECTION_DIST:
        self.detect[i] = self.MIN_DETECTION_DIST
      elif dist > self.MAX_DETECTION_DIST or detection_state == False:
        self.detect[i] = self.MAX_DETECTION_DIST
      else:
        self.detect[i] = self.MAX_DETECTION_DIST - ((dist - self.MAX_DETECTION_DIST) / (self.MIN_DETECTION_DIST - self.MAX_DETECTION_DIST))


  def print_info(self):
    print('Цель: ' + self.TARGET)
    print('Робот: ' + self.ROBOT)
    print('Скорость: ' + str(self.SPEED))


  def tick(self):
    # Задержка программы
    time.sleep(self.SLEEP_TIME)


  def loop(self):
    self._init_values()
    self.read_values()

    distance_to_target = Utils.distance_between_points(self.bot_pos, self.target_pos)
    # Основной цикл программы
    while self.current_dist_to_target_predict > 0.4 or self.current_dist_to_target_predict == 0:
      self.tick()

      # Останавливаем движение и читаем данные робота и цели
      self.stop_move()
      self.read_values()

      # Получаем данные с сенсоров
      self.read_from_sensors()

      # Инициализируем новый кватернион
      q_rot = Quaternion()
      # Получаем кватернион из угла разворота вокруг оси z
      q_rot.set_from_vector(self.bot_euler_angles.z, Vector3(0, 0, 1))
      # Присваиваем вектору робота значение поворота вектора кватернионом
      self.bot_dir = q_rot.rotate(Vector3(1, 0, 0))

      # Проверка текущего состояния и выполнение соответствующей функции
      if self.state == States.MOVING:
        self.action_moving()
      elif self.state == States.ROTATING:
        self.action_rotating()
      elif self.state == States.ROUNDING:
        self.action_rounding()

    print('Цель достигнута')
    self.stop_move()


  def action_moving(self):
    # Если робот оказалася близко к препятствию, объезжаем это препятствие, иначе продолжаем движение до цели
    if (self.detect[4] + self.detect[5])/2 < self.INDENT_DIST:
      # Инициализируем новый кватернион
      q = Quaternion()
      # Получаем кватернион из угла разворота вокруг вектора оси z
      q.set_from_vector(self.PI/2, Vector3(0, 0, 1))
      # Присваиваем вектору цели значение поворота вектора направления робота кватернионом
      self.target_dir = q.rotate(self.bot_dir)
      # Переключаем состояние
      self.state = States.ROTATING
    else:
      # Значение угла в радианах между векторами направления движения робота и между роботом и целью
      angle = Utils.angle_between_vectors(self.bot_dir, self.target_pos.minus(self.bot_pos))

      # Корректируем движение робота до прямолинейного до цели, если необходимо
      if math.fabs(angle) > self.PI/180:
        vrep.simxSetJointTargetVelocity(self.client_id, self.left_motor_handle,  self.SPEED + angle, vrep.simx_opmode_streaming)
        vrep.simxSetJointTargetVelocity(self.client_id, self.right_motor_handle, self.SPEED - angle, vrep.simx_opmode_streaming)
      else:
        vrep.simxSetJointTargetVelocity(self.client_id, self.left_motor_handle,  self.SPEED, vrep.simx_opmode_streaming)
        vrep.simxSetJointTargetVelocity(self.client_id, self.right_motor_handle, self.SPEED, vrep.simx_opmode_streaming)


  def action_rotating(self):
    # Значение угла в радианах между векторами направления движения робота и цели
    angle = Utils.angle_between_vectors(self.bot_dir, self.target_dir)

    # Поворачиваем робота вокруг своей оси, если необходимо, иначе переключаем состояние
    if math.fabs(angle) > 5 * self.PI/180:
      vrep.simxSetJointTargetVelocity(self.client_id, self.left_motor_handle, angle, vrep.simx_opmode_streaming)
      vrep.simxSetJointTargetVelocity(self.client_id, self.right_motor_handle, -angle, vrep.simx_opmode_streaming)
    else:
      self.state = States.ROUNDING


  def action_rounding(self):
    # Инициализация нового кватерниона
    q = Quaternion()
    # Получаем кватернион из угла разворота вокруг оси z
    q.set_from_vector(self.PI/2, Vector3(0, 0, 1))
    # Присваиваем перпендикуляру вектора робота значения поворота вектора направления робота кватернионом
    perp_bot_dir = q.rotate(self.bot_dir)

    # Значение угла в радианах между векторами перпендикуляра к роботу и между роботом и целью
    angle = Utils.angle_between_vectors(perp_bot_dir, self.target_pos.minus(self.bot_pos))

    # Запишем данные angle в файл
    f1 = open('f1.txt', 'a')
    z = str(angle)
    f1.write(z+'\n')
    self.angle_arr.append(angle)

    # Текущая дистанция до цели
    current_dist_to_target_real = Utils.distance_between_points(self.bot_pos, self.target_pos)
    self.current_dist_to_target_predict = data_predict[len(self.angle_arr)]
    print('Predict: ', self.current_dist_to_target_predict)
    print('Real: ', current_dist_to_target_real)
    print('')

    # Запишем данные current_dist_to_target_real в файл
    f2 = open('f2.txt', 'a')
    m = str(current_dist_to_target_real)
    f2.write(m+'\n')

    # Если минимальное значение дистанции до цели меньше, чем текущее, то присваиваем минимальному значению текущее
    if self.min_dist_to_target == None or self.min_dist_to_target <= self.current_dist_to_target_predict:
      self.min_dist_to_target = self.current_dist_to_target_predict
    # Прекращаем объезд препятствия
    elif math.fabs(angle) < 5.0 / 180 * self.PI:
      self.state = States.MOVING
      return

    # Разница между значениями двух крайних справа датчиков
    delta = self.detect[7] - self.detect[8]

    # Устанавливаем расстояние до препятствия
    obstacle_dist = min([self.detect[7], self.detect[8]]) - self.INDENT_DIST

    obstacle_dist_stab = self.obstacle_dist_stab_PID.output(obstacle_dist)
    obstacle_follower = self.obstacle_follower_PID.output(delta)

    obstacle_front = (self.detect[4] + self.detect[5])/2
  
    vrep.simxSetJointTargetVelocity(self.client_id, self.left_motor_handle, self.SPEED + obstacle_follower + obstacle_dist_stab - (1 - obstacle_front), vrep.simx_opmode_streaming)
    vrep.simxSetJointTargetVelocity(self.client_id, self.right_motor_handle, self.SPEED - obstacle_follower - obstacle_dist_stab + (1 - obstacle_front), vrep.simx_opmode_streaming)


bug = Algorithm(ROBOT='P3DX', TARGET='Destination')

bug.loop()